# Install Required Packages

This cell installs all necessary libraries for the ensemble approach:
- PyTorch for deep learning
- MONAI for medical imaging (Swin-UNETR, MedNeXt)
- nnU-Net v2 for automatic segmentation
- Standard ML libraries (numpy, opencv, sklearn, etc.)

**Note:** Adjust CUDA version based on your GPU
**Estimated time:** 5-10 minutes

In [47]:
# Install PyTorch (CUDA 11.8 - adjust for your GPU)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install MONAI with all dependencies
!pip install "monai[all]==1.3.0"

# Install nnU-Net v2
!pip install nnunetv2

# Install other dependencies
!pip install opencv-python scikit-learn pandas matplotlib seaborn tqdm
!pip install SimpleITK nibabel pydicom albumentations

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached monai-1.3.0-202310121228-py3-none-any.whl.metadata (10 kB)
  Using cached clearml-2.1.3-py2.py3-none-any.whl.metadata (17 kB)
  Using cached cucim-23.10.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (43 kB)
  Using cached fire-0.7.1-py3-none-any.whl.metadata (5.8 kB)
  Using cached itk-5.4.5-cp311-abi3-manylinux_2_28_x86_64.whl.metadata (22 kB)
  Using cached lmdb-1.7.5-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (1.4 kB)
  Using cached lpips-0.1.4-py3-none-any.whl.metadata (10 kB)
  Using cached mlflow-3.9.0-py3-none-any.whl.metadata (31 kB)
  Using cached nni-3.0-py3-none-manylinux1_x86_64.whl.metadata (19 kB)
  Using cached openslide_python-1.1.2-cp312-cp312-linux_x86_64.whl
  Using cached pynrrd-1.1.3-py3-none-any.whl.metadata (5.4 kB)
  Using cached pytorch_ignite-0.4.11-py3-none-any.whl.metadata (28 kB)
  Using cached tensorboardx-2.6.4-py3-

Import Libraries

In [48]:
!pip install monai


# Import Required Libraries

Import all necessary libraries for:
- Deep learning (PyTorch, MONAI)
- Data processing (numpy, opencv)
- Visualization (matplotlib, seaborn)
- File handling (pathlib, json)

**No errors should appear if installation was successful**

In [49]:
# Standard libraries
import os
import sys
import json
import warnings
from pathlib import Path
from tqdm import tqdm
import shutil

# Data processing
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import KFold

# Deep learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR

# MONAI
from monai.networks.nets import SwinUNETR, UNet
from monai.losses import DiceLoss
from monai.metrics import DiceMetric
from monai.transforms import (
    Compose, RandRotate, RandFlip, RandZoom
)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

✓ All libraries imported successfully!
PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4


Set Random Seeds

# Set Random Seeds for Reproducibility

Ensures reproducible results across multiple runs by setting:
- Python random seed
- NumPy random seed  
- PyTorch random seed (CPU & GPU)

**Important:** Same seeds = same results

In [50]:
def set_seed(seed=42):
    """Set random seeds for reproducibility"""
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    print(f"✓ Random seed set to {seed}")

set_seed(42)

✓ Random seed set to 42


# Create Project Directory Structure

Creates all necessary folders for:
- Raw data (KSSD2025 images and masks)
- Preprocessed data (nnU-Net and MONAI formats)
- Model checkpoints (saved weights)
- Results (predictions, visualizations)

**Run this before any training**

In [51]:
def create_directories():
    """Create all necessary directories"""
    
    directories = [
        # Data directories
        "data/KSSD2025/images",
        "data/KSSD2025/masks",
        "data/nnUNet_raw",
        "data/nnUNet_preprocessed",
        "data/nnUNet_results",
        "data/MONAI_data",
        
        # Results directories
        "results/nnunet",
        "results/swin_unetr",
        "results/mednext",
        "results/ensemble",
        "results/visualizations",
        
        # Model checkpoints
        "checkpoints/nnunet",
        "checkpoints/swin_unetr",
        "checkpoints/mednext",
    ]
    
    for directory in directories:
        Path(directory).mkdir(parents=True, exist_ok=True)
    
    print("✓ Directory structure created successfully!")
    print(f"  Total directories: {len(directories)}")

create_directories()

✓ Directory structure created successfully!
  Total directories: 14


In [52]:
import os

BASE_PATH = "/kaggle/input/kssd2025-kidney-stone-segmentation-dataset"

for root, dirs, files in os.walk(BASE_PATH):
    print(root)
    print("  Dirs:", dirs)
    print("  Files:", files[:5])
    break


/kaggle/input/kssd2025-kidney-stone-segmentation-dataset
  Dirs: ['data']
  Files: []


# KSSD2025 Dataset Preprocessor

This class handles:
- Creating 5-fold cross-validation splits
- Converting data to nnU-Net format (for nnU-Net model)
- Converting data to MONAI format (for Swin-UNETR and MedNeXt)
- Maintaining consistent splits across all models

**Key Features:**
- Automatic fold splitting
- Format conversion for different frameworks
- Preserves data integrity

# nnU-Net Training Wrapper

nnU-Net is a self-configuring framework that:
- Automatically determines preprocessing
- Automatically selects architecture
- Automatically tunes hyperparameters

This wrapper simplifies:
- Environment setup
- Training all 5 folds
- Running predictions
- Ensemble predictions

**nnU-Net Advantages:**
- Proven SOTA performance
- No manual hyperparameter tuning
- Robust to different datasets

In [45]:
import subprocess

class nnUNetTrainer:
    """
    Wrapper for nnU-Net v2 training
    
    nnU-Net is a self-configuring segmentation method that automatically:
    - Configures preprocessing
    - Selects network architecture  
    - Tunes hyperparameters
    
    We just need to provide the data and let it run!
    """
    
    def __init__(self, dataset_id=500, dataset_name="KSSD2025"):
        """
        Initialize nnU-Net trainer
        
        Args:
            dataset_id: Unique dataset identifier (default: 500)
            dataset_name: Dataset name (default: KSSD2025)
        """
        self.dataset_id = dataset_id
        self.dataset_name = dataset_name
        
        # Set nnU-Net environment variables
        self.setup_environment()
    
    def setup_environment(self):
        """
        Set up nnU-Net environment variables
        
        nnU-Net requires 3 paths:
        - nnUNet_raw: Raw dataset location
        - nnUNet_preprocessed: Where preprocessed data goes
        - nnUNet_results: Where trained models are saved
        """
        base_path = Path("data")
        
        os.environ['nnUNet_raw'] = str(base_path / "nnUNet_raw")
        os.environ['nnUNet_preprocessed'] = str(base_path / "nnUNet_preprocessed")
        os.environ['nnUNet_results'] = str(base_path / "nnUNet_results")
        
        print("✓ nnU-Net environment variables set:")
        print(f"  nnUNet_raw: {os.environ['nnUNet_raw']}")
        print(f"  nnUNet_preprocessed: {os.environ['nnUNet_preprocessed']}")
        print(f"  nnUNet_results: {os.environ['nnUNet_results']}")
    
    def plan_and_preprocess(self):
        """
        Run nnU-Net preprocessing pipeline
        
        This step:
        - Analyzes dataset properties
        - Determines optimal preprocessing
        - Configures network architecture
        - Prepares data for training
        
        **Run this ONCE before training**
        **Time:** ~10-20 minutes
        """
        print("\n=== nnU-Net Planning and Preprocessing ===")
        
        cmd = [
            "nnUNetv2_plan_and_preprocess",
            "-d", str(self.dataset_id),
            "--verify_dataset_integrity"
        ]
        
        print(f"Running: {' '.join(cmd)}")
        subprocess.run(cmd, check=True)
        print("✓ Preprocessing completed!")
    
    def train_fold(self, fold=0, configuration="2d", trainer="nnUNetTrainer"):
        """
        Train a single fold
        
        Args:
            fold: Fold number (0-4)
            configuration: '2d' or '3d' (use 2d for our dataset)
            trainer: nnU-Net trainer variant (default is fine)
            
        **Time per fold:** ~4-6 hours on RTX 3060
        """
        print(f"\n=== Training nnU-Net Fold {fold} ===")
        
        cmd = [
            "nnUNetv2_train",
            str(self.dataset_id),
            configuration,
            str(fold),
            "-tr", trainer,
            "--npz"  # Save probability maps
        ]
        
        print(f"Running: {' '.join(cmd)}")
        subprocess.run(cmd, check=True)
        print(f"✓ Fold {fold} training completed!")
    
    def train_all_folds(self, n_folds=5, configuration="2d"):
        """
        Train all 5 folds sequentially
        
        Args:
            n_folds: Number of folds (default: 5)
            configuration: 2d or 3d
            
        **Total time:** ~20-30 hours (let it run overnight)
        """
        print(f"\n{'='*60}")
        print(f"Training nnU-Net - All {n_folds} Folds")
        print(f"{'='*60}\n")
        
        for fold in range(n_folds):
            self.train_fold(fold=fold, configuration=configuration)
        
        print(f"\n{'='*60}")
        print("✓ All folds training completed!")
        print(f"{'='*60}\n")
    
    def predict(self, input_folder, output_folder, fold=0, configuration="2d"):
        """
        Run prediction on test data using single fold
        
        Args:
            input_folder: Folder with test images
            output_folder: Where to save predictions
            fold: Which fold's model to use
            configuration: 2d or 3d
        """
        print(f"\n=== Running Prediction (Fold {fold}) ===")
        
        cmd = [
            "nnUNetv2_predict",
            "-i", str(input_folder),
            "-o", str(output_folder),
            "-d", str(self.dataset_id),
            "-c", configuration,
            "-f", str(fold),
            "--save_probabilities"
        ]
        
        print(f"Running: {' '.join(cmd)}")
        subprocess.run(cmd, check=True)
        print("✓ Prediction completed!")
    
    def predict_ensemble(self, input_folder, output_folder, 
                        n_folds=5, configuration="2d"):
        """
        Run ensemble prediction across all folds
        
        nnU-Net automatically averages predictions from all folds
        
        Args:
            input_folder: Folder with test images
            output_folder: Where to save predictions
            n_folds: Number of folds to ensemble
            configuration: 2d or 3d
        """
        print(f"\n=== Running nnU-Net Ensemble Prediction ===")
        
        fold_str = " ".join([str(i) for i in range(n_folds)])
        
        cmd = [
            "nnUNetv2_predict",
            "-i", str(input_folder),
            "-o", str(output_folder),
            "-d", str(self.dataset_id),
            "-c", configuration,
            "-f", fold_str,
            "--save_probabilities"
        ]
        
        print(f"Running: {' '.join(cmd)}")
        subprocess.run(cmd, check=True)
        print("✓ Ensemble prediction completed!")

print("✓ nnUNetTrainer class defined")

✓ nnUNetTrainer class defined


In [53]:
import os
import json
import shutil
from pathlib import Path
from sklearn.model_selection import KFold
import numpy as np
import subprocess

# =========================================================
# PART 1: PREPROCESSING
# =========================================================

class KSSD2025Preprocessor:
    def __init__(self, raw_data_path, output_base_path, n_folds=5):
        self.raw_path = Path(raw_data_path)
        self.output_path = Path(output_base_path)
        self.n_folds = n_folds
        
        self.nnunet_path = self.output_path / "nnUNet_raw"
        self.monai_path = self.output_path / "MONAI_data"
        
        self.images = None
        self.masks = None
        
        print(f"✓ Preprocessor initialized")
        print(f"  Raw data: {self.raw_path}")
    
    def _find_images_and_masks(self):
        if self.images is not None and self.masks is not None:
            return self.images, self.masks
            
        img_dir = self.raw_path / "image"
        mask_dir = self.raw_path / "label"
        
        print(f"\n=== Searching for images and masks ===")
        print(f"Image dir: {img_dir}")
        print(f"Mask dir: {mask_dir}")
        
        # .tif files (YOUR dataset format)
        self.images = sorted(list(img_dir.glob("*.tif")))
        self.masks = sorted(list(mask_dir.glob("*.tif")))
        
        print(f"✓ Found {len(self.images)} images and {len(self.masks)} masks")
        
        if len(self.images) == 0:
            raise ValueError(f"No images found in {img_dir}")
        
        return self.images, self.masks
    
    def prepare_nnunet_format(self, dataset_id=500, dataset_name="KSSD2025"):
        print("\n=== Preparing nnU-Net Format ===")
        
        images, masks = self._find_images_and_masks()
        
        dataset_folder = self.nnunet_path / f"Dataset{dataset_id:03d}_{dataset_name}"
        imagesTr = dataset_folder / "imagesTr"
        labelsTr = dataset_folder / "labelsTr"
        
        if dataset_folder.exists():
            print(f"Removing existing dataset...")
            shutil.rmtree(dataset_folder)
        
        imagesTr.mkdir(parents=True, exist_ok=True)
        labelsTr.mkdir(parents=True, exist_ok=True)
        
        print(f"Converting {len(images)} files...")
        
        for idx, (img_path, mask_path) in enumerate(zip(images, masks)):
            case_id = f"{dataset_name}_{idx:04d}"
            shutil.copy(img_path, imagesTr / f"{case_id}_0000.tif")
            shutil.copy(mask_path, labelsTr / f"{case_id}.tif")
            
            if (idx + 1) % 200 == 0:
                print(f"  {idx + 1}/{len(images)} done")
        
        print(f"✓ All {len(images)} files copied")
        
        # nnU-Net v2 dataset.json
        dataset_json = {
            "channel_names": {"0": "grayscale"},
            "labels": {"background": 0, "kidney_stone": 1},
            "numTraining": len(images),
            "file_ending": ".tif",
            "overwrite_image_reader_writer": "NaturalImage2DIO"
        }
        
        with open(dataset_folder / "dataset.json", "w") as f:
            json.dump(dataset_json, f, indent=4)
        
        print(f"✓ dataset.json created")
        return dataset_folder
    
    def prepare_monai_format(self):
        print("\n=== Preparing MONAI Format ===")
        
        images, masks = self._find_images_and_masks()
        
        if len(images) == 0:
            raise ValueError("No images for MONAI!")
        
        self.monai_path.mkdir(parents=True, exist_ok=True)
        
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=42)
        indices = np.arange(len(images))
        
        fold_splits = {}
        for fold, (train_idx, val_idx) in enumerate(kfold.split(indices)):
            fold_splits[f"fold_{fold}"] = {"train": [], "val": []}
            
            for idx in train_idx:
                fold_splits[f"fold_{fold}"]["train"].append({
                    "image": str(images[idx]),
                    "label": str(masks[idx])
                })
            
            for idx in val_idx:
                fold_splits[f"fold_{fold}"]["val"].append({
                    "image": str(images[idx]),
                    "label": str(masks[idx])
                })
            
            print(f"  Fold {fold}: {len(train_idx)} train, {len(val_idx)} val")
        
        with open(self.monai_path / "fold_splits.json", "w") as f:
            json.dump(fold_splits, f, indent=4)
        
        print(f"✓ MONAI ready: {self.n_folds} folds")
        return self.monai_path

# =========================================================
# PART 2: nnU-Net TRAINER
# =========================================================

class nnUNetTrainer:
    def __init__(self, dataset_id=500, dataset_name="KSSD2025", base_path="/kaggle/working"):
        self.dataset_id = dataset_id
        self.dataset_name = dataset_name
        self.base_path = Path(base_path)
        
        self.nnunet_raw = self.base_path / "nnUNet_raw"
        self.nnunet_preprocessed = self.base_path / "nnUNet_preprocessed"
        self.nnunet_results = self.base_path / "nnUNet_results"
        
        self.nnunet_preprocessed.mkdir(parents=True, exist_ok=True)
        self.nnunet_results.mkdir(parents=True, exist_ok=True)
        
        os.environ['nnUNet_raw'] = str(self.nnunet_raw)
        os.environ['nnUNet_preprocessed'] = str(self.nnunet_preprocessed)
        os.environ['nnUNet_results'] = str(self.nnunet_results)
        
        print(f"✓ nnU-Net environment set")
    
    def plan_and_preprocess(self):
        print(f"\n=== nnU-Net Planning and Preprocessing ===")
        
        # IMPORTANT: NO --verify_dataset_integrity flag
        cmd = ["nnUNetv2_plan_and_preprocess", "-d", str(self.dataset_id)]
        
        print(f"Running: {' '.join(cmd)}")
        print("⏳ This takes 5-15 minutes...")
        subprocess.run(cmd, check=True)
        print("✓ Preprocessing done!")
    
    def train_fold(self, fold=0, configuration="2d"):
        print(f"\n=== Training Fold {fold} ===")
        cmd = ["nnUNetv2_train", str(self.dataset_id), configuration, str(fold)]
        print(f"Running: {' '.join(cmd)}")
        subprocess.run(cmd, check=True)
        print(f"✓ Fold {fold} done!")
    
    def train_all_folds(self, n_folds=5, configuration="2d"):
        print(f"\n{'='*60}")
        print(f"⚠️  Training {n_folds} folds (24-48 hours!)")
        print(f"{'='*60}\n")
        
        for fold in range(n_folds):
            self.train_fold(fold, configuration)
        
        print(f"\n✓ All {n_folds} folds trained!")

# =========================================================
# RUN EVERYTHING
# =========================================================

print("="*60)
print("PREPROCESSING")
print("="*60)

preprocessor = KSSD2025Preprocessor(
    raw_data_path="/kaggle/input/kssd2025-kidney-stone-segmentation-dataset/data",
    output_base_path="/kaggle/working",
    n_folds=5
)

nnunet_folder = preprocessor.prepare_nnunet_format(dataset_id=500, dataset_name="KSSD2025")
monai_folder = preprocessor.prepare_monai_format()

print("\n" + "="*60)
print("nnU-Net SETUP")
print("="*60)

nnunet_trainer = nnUNetTrainer(dataset_id=500, base_path="/kaggle/working")
nnunet_trainer.plan_and_preprocess()

print("\n" + "="*60)
print("✓✓✓ SETUP COMPLETE! ✓✓✓")
print("="*60)
print(f"nnU-Net: {nnunet_folder}")
print(f"MONAI: {monai_folder}")
print(f"Samples: 838 (.tif files)")
print("\nTo train (24-48 hrs):")
print("  nnunet_trainer.train_all_folds(n_folds=5)")
print("="*60)

PREPROCESSING
✓ Preprocessor initialized
  Raw data: /kaggle/input/kssd2025-kidney-stone-segmentation-dataset/data

=== Preparing nnU-Net Format ===

=== Searching for images and masks ===
Image dir: /kaggle/input/kssd2025-kidney-stone-segmentation-dataset/data/image
Mask dir: /kaggle/input/kssd2025-kidney-stone-segmentation-dataset/data/label
✓ Found 838 images and 838 masks
Removing existing dataset...
Converting 838 files...
  200/838 done
  400/838 done
  600/838 done
  800/838 done
✓ All 838 files copied
✓ dataset.json created

=== Preparing MONAI Format ===
  Fold 0: 670 train, 168 val
  Fold 1: 670 train, 168 val
  Fold 2: 670 train, 168 val
  Fold 3: 671 train, 167 val
  Fold 4: 671 train, 167 val
✓ MONAI ready: 5 folds

nnU-Net SETUP
✓ nnU-Net environment set

=== nnU-Net Planning and Preprocessing ===
Running: nnUNetv2_plan_and_preprocess -d 500
⏳ This takes 5-15 minutes...
Fingerprint extraction...
Dataset500_KSSD2025
Experiment planning...

############################
INFO

100%|██████████| 838/838 [01:31<00:00,  9.18it/s]


Configuration: 3d_fullres...
INFO: Configuration 3d_fullres not found in plans file nnUNetPlans.json of dataset Dataset500_KSSD2025. Skipping.
Configuration: 3d_lowres...
INFO: Configuration 3d_lowres not found in plans file nnUNetPlans.json of dataset Dataset500_KSSD2025. Skipping.
✓ Preprocessing done!

✓✓✓ SETUP COMPLETE! ✓✓✓
nnU-Net: /kaggle/working/nnUNet_raw/Dataset500_KSSD2025
MONAI: /kaggle/working/MONAI_data
Samples: 838 (.tif files)

To train (24-48 hrs):
  nnunet_trainer.train_all_folds(n_folds=5)


training 

In [54]:
# Train all 5 folds
# ⚠️ WARNING: This takes 24-48 hours!
nnunet_trainer.train_all_folds(n_folds=5, configuration="2d")

print("\n" + "="*60)
print("✓ nnU-Net training completed!")
print("  Check results in: /kaggle/working/nnUNet_results/")
print("="*60)


⚠️  Training 5 folds (24-48 hours!)


=== Training Fold 0 ===
Running: nnUNetv2_train 500 2d 0

############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2026-02-07 21:10:00.857970: Using torch.compile...
2026-02-07 21:10:04.468783: do_dummy_2d_data_aug: False
2026-02-07 21:10:04.473538: Creating new 5-fold cross-validation split...
2026-02-07 21:10:04

/usr/local/lib/python3.12/dist-packages/fft_conv_pytorch/fft_conv.py:139: UserWarning: Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:306.)
  output = output[crop_slices].contiguous()
/usr/local/lib/python3.12/dist-packages/fft_conv_pytorch/fft_conv.py:139: UserWarning: Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:306.)
  output = output[crop_slices].contiguou

using pin_memory on device 0


W0207 21:10:19.886000 14686 torch/_inductor/utils.py:1436] [0/0] Not enough SMs to use max_autotune_gemm mode
/usr/local/lib/python3.12/dist-packages/torch/_inductor/lowering.py:7095: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(
/tmp/torchinductor_root/4b/c4bn6mzsaechnaq3f33zbt5wcpz6imzscruterv2mselskedgufv.py:28: unknown: block: [332,0,0], thread: [106,0,0] Assertion `index out of bounds: 0 <= tmp1 < 2` failed.
/tmp/torchinductor_root/4b/c4bn6mzsaechnaq3f33zbt5wcpz6imzscruterv2mselskedgufv.py:28: unknown: block: [332,0,0], thread: [107,0,0] Assertion `index out of bounds: 0 <= tmp1 < 2` failed.
/tmp/torchinductor_root/4b/c4bn6mzsaechnaq3f33zbt5wcpz6imzscruterv2mselskedgufv.py:28: unknown: block: [1420,0,0], thread: [102,0,0] Assertion `index out of bounds: 0 <= tmp1 < 2` failed.
/tmp/torchinductor_root/4b/c4bn

using pin_memory on device 0

This is the configuration used by this training:
Configuration name: 2d
 {'data_identifier': 'nnUNetPlans_2d', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 14, 'patch_size': [448, 512], 'median_image_size_in_voxels': [416.0, 512.0], 'spacing': [1.0, 1.0], 'normalization_schemes': ['ZScoreNormalization'], 'use_mask_for_norm': [False], 'resampling_fn_data': 'resample_data_or_seg_to_shape', 'resampling_fn_seg': 'resample_data_or_seg_to_shape', 'resampling_fn_data_kwargs': {'is_seg': False, 'order': 3, 'order_z': 0, 'force_separate_z': None}, 'resampling_fn_seg_kwargs': {'is_seg': True, 'order': 1, 'order_z': 0, 'force_separate_z': None}, 'resampling_fn_probabilities': 'resample_data_or_seg_to_shape', 'resampling_fn_probabilities_kwargs': {'is_seg': False, 'order': 1, 'order_z': 0, 'force_separate_z': None}, 'architecture': {'network_class_name': 'dynamic_network_architectures.architectures.unet.PlainConvUNet', 'arch_kwargs': {'n_stages': 7, 'feat

Exception in thread Thread-3 (results_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.12/dist-packages/batchgenerators/dataloading/nondet_multi_threaded_augmenter.py", line 125, in results_loop
    raise e
  File "/usr/local/lib/python3.12/dist-packages/batchgenerators/dataloading/nondet_multi_threaded_augmenter.py", line 103, in results_loop
    raise RuntimeError("One or more background workers are no longer alive. Exiting. Please check the "
RuntimeError: One or more background workers are no longer alive. Exiting. Please check the print statements above for the actual error message
Exception in thread Thread-2 (results_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/

CalledProcessError: Command '['nnUNetv2_train', '500', '2d', '0']' died with <Signals.SIGABRT: 6>.